In [38]:
import logging
from googleapiclient.discovery import build
from google.colab import auth
from google.auth import default
from googleapiclient.errors import HttpError

In [40]:
auth.authenticate_user()
logging.basicConfig(level=logging.INFO)

In [ ]:
def initialize_api():
    """
    Initializes the Google Docs and Drive API using OAuth2 credentials.
    Returns:
        googleapiclient.discovery.Resource: Authenticated service object for the API.
    """
    SCOPES = ['https://www.googleapis.com/auth/documents', 'https://www.googleapis.com/auth/drive.file']
    credentials, _ = default(scopes=SCOPES)
    return build('docs', 'v1', credentials=credentials)

In [ ]:
docs_service = initialize_api()

In [ ]:
markdown_content = """
# Product Team Sync - May 15, 2023

## Attendees
- Sarah Chen (Product Lead)
- Mike Johnson (Engineering)
- Anna Smith (Design)
- David Park (QA)

## Agenda

### 1. Sprint Review
* Completed Features
  * User authentication flow
  * Dashboard redesign
  * Performance optimization
    * Reduced load time by 40%
    * Implemented caching solution
* Pending Items
  * Mobile responsive fixes
  * Beta testing feedback integration

### 2. Current Challenges
* Resource constraints in QA team
* Third-party API integration delays
* User feedback on new UI
  * Navigation confusion
  * Color contrast issues

### 3. Next Sprint Planning
* Priority Features
  * Payment gateway integration
  * User profile enhancement
  * Analytics dashboard
* Technical Debt
  * Code refactoring
  * Documentation updates

## Action Items
- [ ] @sarah: Finalize Q3 roadmap by Friday
- [ ] @mike: Schedule technical review for payment integration
- [ ] @anna: Share updated design system documentation
- [ ] @david: Prepare QA resource allocation proposal

## Next Steps
* Schedule individual team reviews
* Update sprint board
* Share meeting summary with stakeholders

## Notes
* Next sync scheduled for May 22, 2023
* Platform demo for stakeholders on May 25
* Remember to update JIRA tickets

Meeting recorded by: Sarah Chen
Duration: 45 minutes
"""

In [ ]:
def parse_markdown_to_google_docs(markdown_content, doc_id):
    try:
      lines = markdown_content.splitlines()
      requests = []
      current_index = 1  # Start index for writing in the document

      for line in lines:
          line = line.strip()
          if line.startswith("# "):
              requests.append({
                  "insertText": {
                      "location": {"index": current_index},
                      "text": line[2:] + "\n"
                  }
              })
              requests.append({
                  "updateParagraphStyle": {
                      "range": {
                          "startIndex": current_index,
                          "endIndex": current_index + len(line[2:]) + 1
                      },
                      "paragraphStyle": {"namedStyleType": "HEADING_1"},
                      "fields": "namedStyleType"
                  }
              })
              current_index += len(line[2:]) + 1
              continue

          # Handle Heading 2 (##)
          elif line.startswith("## "):
              requests.append({
                  "insertText": {
                      "location": {"index": current_index},
                      "text": line[3:] + "\n"
                  }
              })
              requests.append({
                  "updateParagraphStyle": {
                      "range": {
                          "startIndex": current_index,
                          "endIndex": current_index + len(line[3:]) + 1
                      },
                      "paragraphStyle": {"namedStyleType": "HEADING_2"},
                      "fields": "namedStyleType"
                  }
              })
              current_index += len(line[3:]) + 1
              continue

          # Handle Heading 3 (###)
          elif line.startswith("### "):
              requests.append({
                  "insertText": {
                      "location": {"index": current_index},
                      "text": line[4:] + "\n"
                  }
              })
              requests.append({
                  "updateParagraphStyle": {
                      "range": {
                          "startIndex": current_index,
                          "endIndex": current_index + len(line[4:]) + 1
                      },
                      "paragraphStyle": {"namedStyleType": "HEADING_3"},
                      "fields": "namedStyleType"
                  }
              })
              current_index += len(line[4:]) + 1
              continue

          # Handle bullet points (*)
          elif line.startswith(("* ")):
              requests.append({
                  "insertText": {
                      "location": {"index": current_index},
                      "text": line[2:] + "\n"
                  }
              })
              requests.append({
                  "createParagraphBullets": {
                      "range": {
                          "startIndex": current_index,
                          "endIndex": current_index + len(line[2:]) + 1
                      },
                      "bulletPreset": "BULLET_DISC_CIRCLE_SQUARE"  # Valid bullet preset
                  }
              })
              current_index += len(line[2:]) + 1
              continue

          # Handle checkboxes (- [ ])
          elif line.startswith("- [ ]"):
              content = line[6:]  # Remove "- [ ] " from the line
              # Extract assignee mention if present
              if "@" in content:
                  parts = content.split(":")
                  assignee = parts[0].strip()  # Assignee (e.g., "@sarah")
                  task = parts[1].strip() if len(parts) > 1 else ""

                  # Add the assignee in bold and task normally
                  requests.append({
                      "insertText": {
                          "location": {"index": current_index},
                          "text": f"{assignee}: {task}\n"
                      }
                  })
                  requests.append({
                      "updateTextStyle": {
                          "range": {
                              "startIndex": current_index,
                              "endIndex": current_index + len(assignee)
                          },
                          "textStyle": {"bold": True, "foregroundColor": {"color": {"rgbColor": {"red": 0.0, "green": 0.5, "blue": 1.0}}}},
                          "fields": "bold,foregroundColor"
                      }
                  })
                  current_index += len(assignee) + len(task) + 3  # Update index
              else:
                  requests.append({
                      "insertText": {
                          "location": {"index": current_index},
                          "text": content + "\n"
                      }
                  })
                  current_index += len(content) + 1

              # Add Google Docs checkbox style
              requests.append({
                  "createParagraphBullets": {
                      "range": {
                          "startIndex": current_index - len(content) - 1,
                          "endIndex": current_index
                      },
                      "bulletPreset": "BULLET_CHECKBOX"
                  }
              })

          # Handle footer
          elif line.startswith("Meeting recorded by:") or line.startswith("Duration:"):
              requests.append({
                  "insertText": {
                      "location": {"index": current_index},
                      "text": line + "\n"
                  }
              })
              requests.append({
                  "updateTextStyle": {
                      "range": {
                          "startIndex": current_index,
                          "endIndex": current_index + len(line)
                      },
                      "textStyle": {"italic": True, "foregroundColor": {"color": {"rgbColor": {"red": 0.4, "green": 0.4, "blue": 0.4}}}},
                      "fields": "italic,foregroundColor"
                  }
              })
              current_index += len(line) + 1

          # Other cases: Process normally (e.g., headings, bullet points)
          elif line:
              requests.append({
                  "insertText": {
                      "location": {"index": current_index},
                      "text": line + "\n"
                  }
              })
              current_index += len(line) + 1

      # Send batchUpdate request to Google Docs API
      docs_service.documents().batchUpdate(
          documentId=doc_id, body={"requests": requests}
      ).execute()
      logging.info("Markdown successfully uploaded to Google Docs")

    except HttpError as e:
        logging.error(f"Google API Error: {e}")
    except Exception as e:
        logging.error(f"An error occurred: {e}")

In [ ]:
def create_google_doc(title):
    body = {"title": title}
    doc = docs_service.documents().create(body=body).execute()
    print(f"Created document: {doc['title']} (ID: {doc['documentId']})")
    return doc['documentId']

In [ ]:
doc_title = "Markdown to Google Docs"
doc_id = create_google_doc(doc_title)
parse_markdown_to_google_docs(markdown_content, doc_id)

Created document: Markdown to Google Docs (ID: 1pNXTqBbOj4XFQiRMO48QH3c0heWhuJ0JEOUQRaIttZE)
